### Imports

In [4]:
# Data Handling
import pandas as pd
import numpy as np

# Financial Data
import yfinance as yf

# Technical Analysis
import ta

# Machine Learning
import sklearn

# Visualization
import matplotlib.pyplot as plt

# Backtesting
import backtrader as bt

### Top 100 Stocks over the past 5 years

In [44]:
import yfinance as yf
import pandas as pd

# List of the top 10 ticker symbols
ticker_symbols = [ "MSFT", "AAPL", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "SPY", "V", "AMD",
    "NVDA", "INTC", "CSCO", "WMT", "DIS", "MCD", "BA", "PYPL", "SNAP", "NKE",
    "XOM", "CVX", "JNJ", "PFE", "BABA", "T", "VZ", "GE", "IBM", "GS", "JPM",
    "MS", "GS", "UNH", "CVS", "HD", "LOW", "UPS", "SBUX", "BLK", "DELL", "LMT",
    "ABT", "TXN", "TMO", "MRK", "AMAT", "INTU", "ADI", "MU", "LULU", "PLUG",
    "F", "GM", "COF", "STZ", "KMX", "TSCO", "LMT", "SHOP", "SQ", "BIDU", "SPG",
    "MELI", "BA", "ATVI", "GS", "CAT", "HLT", "TGT", "DOW", "COP", "BA", "NKE",
    "LULU", "GS", "JNJ", "JPM", "BRK-A", "MSFT", "V", "TSLA", "XOM", "PFE",
    "WMT", "GE", "T", "VZ", "AMZN", "BABA", "CSCO", "NVDA", "AAPL", "GOOGL", "INTC",
    "AMT", "CVX", "META", "WFC", "KO", "PEP", "K", "SBUX", "NKE", "LOW", "MCD",
    "DIS", "WMT", "XOM", "UPS", "HD", "TMO", "PYPL", "UBER", "ZM", "TEAM", "MDB",
    "DKNG"]

# Create an empty DataFrame to store all the data
all_data = pd.DataFrame()

# Loop through each ticker symbol and fetch data
for ticker_symbol in ticker_symbols:
    # Create a Ticker object for each stock
    ticker = yf.Ticker(ticker_symbol)

    # Fetch historical market data from 2024 to the present
    historical_data = ticker.history(start="2020-01-01", end=pd.to_datetime("today").strftime('%Y-%m-%d'))

    # Add a new column 'Ticker' with the ticker symbol for each row
    historical_data['Ticker'] = ticker_symbol

    # Append the data to the all_data DataFrame
    all_data = pd.concat([all_data, historical_data])

# Reset the index after combining all the data
all_data.reset_index(inplace=True)

# Display the first few rows of the combined data
print(all_data[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']])

# Optionally, save the data to a CSV file
all_data.to_csv('top_10_stocks_2024.csv', index=False)
print("Data saved to top_10_stocks_2024.csv")

$ATVI: possibly delisted; no timezone found
C:\Users\Sezy\AppData\Local\Temp\ipykernel_33544\2875241900.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_data = pd.concat([all_data, historical_data])


       Ticker                      Date        Open        High         Low  \
0        MSFT 2020-01-02 00:00:00-05:00  151.870762  153.735906  151.440346   
1        MSFT 2020-01-03 00:00:00-05:00  151.430809  152.989871  151.182113   
2        MSFT 2020-01-06 00:00:00-05:00  150.244705  152.176809  149.699501   
3        MSFT 2020-01-07 00:00:00-05:00  152.387267  152.722028  150.474296   
4        MSFT 2020-01-08 00:00:00-05:00  152.014192  153.802829  151.076840   
...       ...                       ...         ...         ...         ...   
149287   DKNG 2025-01-23 00:00:00-05:00   40.770000   41.535000   40.250000   
149288   DKNG 2025-01-24 00:00:00-05:00   41.570000   42.000000   41.090000   
149289   DKNG 2025-01-27 00:00:00-05:00   40.500000   41.455002   40.070000   
149290   DKNG 2025-01-28 00:00:00-05:00   41.250000   42.279999   41.230000   
149291   DKNG 2025-01-29 00:00:00-05:00   42.200001   42.305000   41.200001   

             Close      Volume  
0       153.630692

### UPDATE STOCKS INFO
Ran to update MongoDatabase daily, collecting most recent stock data

In [1]:
from datetime import datetime, timedelta
from pymongo import MongoClient
import yfinance as yf
from dateutil import parser

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["Stock_Data"]
collection = db["Stock_5year"]

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# List of 100 ticker symbols (Your original list)
ticker_symbols = [ 
    "MSFT", "AAPL", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "SPY", "V", "AMD",
    "NVDA", "INTC", "CSCO", "WMT", "DIS", "MCD", "BA", "PYPL", "SNAP", "NKE",
    "XOM", "CVX", "JNJ", "PFE", "BABA", "T", "VZ", "GE", "IBM", "GS", "JPM",
    "MS", "GS", "UNH", "CVS", "HD", "LOW", "UPS", "SBUX", "BLK", "DELL", "LMT",
    "ABT", "TXN", "TMO", "MRK", "AMAT", "INTU", "ADI", "MU", "LULU", "PLUG",
    "F", "GM", "COF", "STZ", "KMX", "TSCO", "LMT", "SHOP", "SQ", "BIDU", "SPG",
    "MELI", "BA", "ATVI", "GS", "CAT", "HLT", "TGT", "DOW", "COP", "BA", "NKE",
    "LULU", "GS", "JNJ", "JPM", "BRK-A", "MSFT", "V", "TSLA", "XOM", "PFE",
    "WMT", "GE", "T", "VZ", "AMZN", "BABA", "CSCO", "NVDA", "AAPL", "GOOGL", "INTC",
    "AMT", "CVX", "META", "WFC", "KO", "PEP", "K", "SBUX", "NKE", "LOW", "MCD",
    "DIS", "WMT", "XOM", "UPS", "HD", "TMO", "PYPL", "UBER", "ZM", "TEAM", "MDB",
    "DKNG"
]

# Loop through each stock and update only missing data
for ticker_symbol in ticker_symbols:
    print(f"🔍 Checking latest data for {ticker_symbol}...")

    # Get the most recent date for this stock in MongoDB
    latest_entry = collection.find_one({"Ticker": ticker_symbol}, sort=[("Date", -1)])

    if latest_entry:
        last_date_in_db = latest_entry["Date"]
        # Check if the last_date_in_db is a string and parse it
        if isinstance(last_date_in_db, str):
            last_date_in_db = parser.parse(last_date_in_db)
    else:
        # If no records exist, start from 5 years ago
        last_date_in_db = datetime.today() - timedelta(days=5*365)

    # Fetch only new data since the last recorded date
    start_date = (last_date_in_db + timedelta(days=1)).strftime('%Y-%m-%d')

    # If start_date is today, no need to fetch (already up to date)
    if start_date >= today:
        print(f"✅ {ticker_symbol} is already up-to-date.")
        continue

    print(f"📈 Fetching new data for {ticker_symbol} from {start_date} to {today}...")

    ticker = yf.Ticker(ticker_symbol)
    new_data = ticker.history(start=start_date, end=today)

    if new_data.empty:
        print(f"⚠️ No new data found for {ticker_symbol}, skipping...")
        continue

    # Reset index & add ticker column
    new_data.reset_index(inplace=True)
    new_data["Ticker"] = ticker_symbol

    # Convert to dictionary format for MongoDB
    records = new_data.to_dict(orient='records')

    # Insert new records into MongoDB
    collection.insert_many(records)
    print(f"✅ Updated {len(records)} new entries for {ticker_symbol}")

print("🎉 Stock data update complete!")

🔍 Checking latest data for MSFT...
📈 Fetching new data for MSFT from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for MSFT
🔍 Checking latest data for AAPL...
📈 Fetching new data for AAPL from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for AAPL
🔍 Checking latest data for GOOGL...
📈 Fetching new data for GOOGL from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for GOOGL
🔍 Checking latest data for AMZN...
📈 Fetching new data for AMZN from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for AMZN
🔍 Checking latest data for TSLA...
📈 Fetching new data for TSLA from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for TSLA
🔍 Checking latest data for META...
📈 Fetching new data for META from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for META
🔍 Checking latest data for NVDA...
📈 Fetching new data for NVDA from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for NVDA
🔍 Checking latest data for SPY...
📈 Fetching new data for SPY from 2025-02-04 to 2025-0

$ATVI: possibly delisted; no timezone found


⚠️ No new data found for ATVI, skipping...
🔍 Checking latest data for GS...
✅ GS is already up-to-date.
🔍 Checking latest data for CAT...
📈 Fetching new data for CAT from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for CAT
🔍 Checking latest data for HLT...
📈 Fetching new data for HLT from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for HLT
🔍 Checking latest data for TGT...
📈 Fetching new data for TGT from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for TGT
🔍 Checking latest data for DOW...
📈 Fetching new data for DOW from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for DOW
🔍 Checking latest data for COP...
📈 Fetching new data for COP from 2025-02-04 to 2025-02-06...
✅ Updated 2 new entries for COP
🔍 Checking latest data for BA...
✅ BA is already up-to-date.
🔍 Checking latest data for NKE...
✅ NKE is already up-to-date.
🔍 Checking latest data for LULU...
✅ LULU is already up-to-date.
🔍 Checking latest data for GS...
✅ GS is already up-to-date.
🔍 Checking 

### Inserting The Past 5 years of data

In [3]:
import pandas as pd
from pymongo import MongoClient
df = pd.read_csv(r'C:\Users\Sezy\OneDrive\Personal_Projects\Trading_Algorithm\CSVFILES\top_10_stocks_2024.csv')

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Default MongoDB URI
db = client["Stock_Data"]  # Replace with your desired database name
collection = db["Stock_5year"]  # Replace with your desired collection name

# Convert DataFrame to dictionary and insert into MongoDB
records = df.to_dict(orient='records')
collection.insert_many(records)

print(f"Inserted {len(records)} records into the database!")

Inserted 149292 records into the database!


In [10]:
import yfinance as yf

ticker = "MSFT"
stock = yf.Ticker(ticker)
options = stock.options  # Get all expiration dates available

# You can get the options chain for a specific expiration
options_chain = stock.option_chain(options[0])
puts = options_chain.puts  # Data for puts
print(puts.head())

        contractSymbol             lastTradeDate  strike  lastPrice  bid  \
0  MSFT250207P00230000 2025-01-28 14:30:12+00:00   230.0       0.02  0.0   
1  MSFT250207P00250000 2025-01-31 15:33:23+00:00   250.0       0.01  0.0   
2  MSFT250207P00270000 2025-01-08 15:56:30+00:00   270.0       0.05  0.0   
3  MSFT250207P00280000 2025-01-24 18:22:45+00:00   280.0       0.01  0.0   
4  MSFT250207P00290000 2025-01-27 20:13:38+00:00   290.0       0.02  0.0   

    ask  change  percentChange  volume  openInterest  impliedVolatility  \
0  0.01     0.0            0.0     2.0             9           1.500002   
1  0.01     0.0            0.0     4.0             4           1.281254   
2  0.01     0.0            0.0     NaN             2           1.093755   
3  0.01     0.0            0.0     5.0             5           1.000005   
4  0.01     0.0            0.0     5.0           107           0.937501   

   inTheMoney contractSize currency  
0       False      REGULAR      USD  
1       False   

In [1]:
import yfinance as yf
import pandas as pd

# Define the stock ticker
ticker = "MSFT"
stock = yf.Ticker(ticker)

# Get available expiration dates
expiration_dates = stock.options
print("Available Expiration Dates:", expiration_dates)

# Select the nearest expiration date
selected_expiration = expiration_dates[0]  # Earliest available expiry

# Get options chain for the selected date
options_chain = stock.option_chain(selected_expiration)
puts = options_chain.puts  # Extract put options data

# Get the current stock price
current_price = stock.history(period="1d")["Close"].iloc[-1]

# Filter for in-the-money (ITM) puts (strike price > current stock price)
itm_puts = puts[puts["strike"] > current_price]

# Display first few rows of the ITM puts
print(itm_puts.head())

Available Expiration Dates: ('2025-02-07', '2025-02-14', '2025-02-21', '2025-02-28', '2025-03-07', '2025-03-14', '2025-03-21', '2025-04-17', '2025-05-16', '2025-06-20', '2025-07-18', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-03-20', '2026-06-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17')
         contractSymbol             lastTradeDate  strike  lastPrice    bid  \
38  MSFT250207P00415000 2025-02-04 16:01:34+00:00   415.0       4.84   4.70   
39  MSFT250207P00417500 2025-02-04 16:00:37+00:00   417.5       6.30   6.30   
40  MSFT250207P00420000 2025-02-04 16:01:34+00:00   420.0       8.04   7.80   
41  MSFT250207P00422500 2025-02-04 15:07:50+00:00   422.5      10.15   9.75   
42  MSFT250207P00425000 2025-02-04 15:47:53+00:00   425.0      12.20  12.00   

      ask  change  percentChange  volume  openInterest  impliedVolatility  \
38   4.80   -2.16     -30.857141   674.0          3858           0.220467   
39   6.45   -2.00     -24.096384    69.0        

In [14]:
import yfinance as yf
from datetime import datetime, timedelta

# List of stock symbols (replace with your actual list of 100 stocks)
ticker_symbols = ["MSFT", "AAPL", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "SPY", "V", "AMD",
    "NVDA", "INTC", "CSCO", "WMT", "DIS", "MCD", "BA", "PYPL", "SNAP", "NKE",
    "XOM", "CVX", "JNJ", "PFE", "BABA", "T", "VZ", "GE", "IBM", "GS", "JPM",
    "MS", "GS", "UNH", "CVS", "HD", "LOW", "UPS", "SBUX", "BLK", "DELL", "LMT",
    "ABT", "TXN", "TMO", "MRK", "AMAT", "INTU", "ADI", "MU", "LULU", "PLUG",
    "F", "GM", "COF", "STZ", "KMX", "TSCO", "LMT", "SHOP", "SQ", "BIDU", "SPG",
    "MELI", "BA", "ATVI", "GS", "CAT", "HLT", "TGT", "DOW", "COP", "BA", "NKE",
    "LULU", "GS", "JNJ", "JPM", "BRK-A", "MSFT", "V", "TSLA", "XOM", "PFE",
    "WMT", "GE", "T", "VZ", "AMZN", "BABA", "CSCO", "NVDA", "AAPL", "GOOGL", "INTC",
    "AMT", "CVX", "META", "WFC", "KO", "PEP", "K", "SBUX", "NKE", "LOW", "MCD",
    "DIS", "WMT", "XOM", "UPS", "HD", "TMO", "PYPL", "UBER", "ZM", "TEAM", "MDB",
    "DKNG"]  # Example stock symbols

# Loop through the list of ticker symbols
for ticker_symbol in ticker_symbols:
    print(f"Processing {ticker_symbol}...")  # Debugging print statement
    stock = yf.Ticker(ticker_symbol)

    # Step 1: Get expiration dates
    expiration_dates = stock.options
    print(f"Expiration Dates for {ticker_symbol}: {expiration_dates}")  # Debugging print statement

    # If expiration_dates is empty, skip this stock
    if not expiration_dates:
        print(f"No options data available for {ticker_symbol}. Skipping...\n")
        continue

    # Step 2: Select the expiration date that is around 3 weeks from today
    today = datetime.today()
    target_date = today + timedelta(weeks=3)

    # Find the closest expiration date
    selected_expiration = min(expiration_dates, key=lambda date: abs(datetime.strptime(date, "%Y-%m-%d") - target_date))
    print(f"Selected Expiration Date for {ticker_symbol}: {selected_expiration}")

    # Step 3: Get option chain for the selected expiration date
    option_chain = stock.option_chain(selected_expiration)
    put_options = option_chain.puts

    # Step 4: Get out-of-the-money (OTM) put options (strike price < current stock price)
    current_stock_price = stock.history(period="1d")["Close"].iloc[-1]
    print(f"Current Stock Price for {ticker_symbol}: {current_stock_price}")

    otm_puts = put_options[put_options["strike"] < current_stock_price]

    # Step 5: Select the 10 closest OTM strike prices
    # Sort by strike price descending (closest to current price)
    sorted_otm_puts = otm_puts.sort_values(by="strike", ascending=False)

    # Get the 10 closest OTM puts in 50-cent increments
    closest_strikes = sorted_otm_puts.head(10)

    # Display the filtered put options
    print(f"10 Closest OTM Put Options for {ticker_symbol}:")
    print(closest_strikes[["strike", "lastPrice", "bid", "ask", "volume", "openInterest"]])
    print("\n" + "="*50 + "\n")

Processing MSFT...
Expiration Dates for MSFT: ('2025-02-07', '2025-02-14', '2025-02-21', '2025-02-28', '2025-03-07', '2025-03-14', '2025-03-21', '2025-04-17', '2025-05-16', '2025-06-20', '2025-07-18', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-03-20', '2026-06-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17')
Selected Expiration Date for MSFT: 2025-02-28
Current Stock Price for MSFT: 412.6700134277344
10 Closest OTM Put Options for MSFT:
    strike  lastPrice   bid   ask  volume  openInterest
25   410.0       7.95  7.70  7.85    93.0          1280
24   405.0       5.92  5.90  6.10    81.0          1604
23   400.0       4.30  4.40  4.50   133.0          1849
22   395.0       3.15  3.15  3.25    58.0          1523
21   390.0       2.29  2.32  2.38    95.0          1236
20   385.0       1.63  1.69  1.75   156.0           807
19   380.0       1.28  1.24  1.28   233.0           561
18   375.0       0.96  0.92  0.96   144.0           564
17   370.0       0.73  0

In [16]:
import yfinance as yf
import pandas as pd

# List of tickers for the 100 stocks
ticker_symbols = ["MSFT", "AAPL", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "SPY", "V", "AMD",
    "NVDA", "INTC", "CSCO", "WMT", "DIS", "MCD", "BA", "PYPL", "SNAP", "NKE",
    "XOM", "CVX", "JNJ", "PFE", "BABA", "T", "VZ", "GE", "IBM", "GS", "JPM",
    "MS", "GS", "UNH", "CVS", "HD", "LOW", "UPS", "SBUX", "BLK", "DELL", "LMT",
    "ABT", "TXN", "TMO", "MRK", "AMAT", "INTU", "ADI", "MU", "LULU", "PLUG",
    "F", "GM", "COF", "STZ", "KMX", "TSCO", "LMT", "SHOP", "SQ", "BIDU", "SPG",
    "MELI", "BA", "ATVI", "GS", "CAT", "HLT", "TGT", "DOW", "COP", "BA", "NKE",
    "LULU", "GS", "JNJ", "JPM", "BRK-A", "MSFT", "V", "TSLA", "XOM", "PFE",
    "WMT", "GE", "T", "VZ", "AMZN", "BABA", "CSCO", "NVDA", "AAPL", "GOOGL", "INTC",
    "AMT", "CVX", "META", "WFC", "KO", "PEP", "K", "SBUX", "NKE", "LOW", "MCD",
    "DIS", "WMT", "XOM", "UPS", "HD", "TMO", "PYPL", "UBER", "ZM", "TEAM", "MDB",
    "DKNG"] 


# Empty list to store options data
options_data = []

# Loop through each ticker symbol to fetch options data
for ticker_symbol in ticker_symbols:
    stock = yf.Ticker(ticker_symbol)
    
    # Step 1: Get expiration dates
    expiration_dates = stock.options
    
    if not expiration_dates:
        print(f"No options available for {ticker_symbol}. Skipping.")
        continue
    
    # Step 2: Select the expiration date that is around 3 weeks from today
    today = datetime.today()
    target_date = today + timedelta(weeks=3)
    
    # Find the closest expiration date if available
    try:
        selected_expiration = min(expiration_dates, key=lambda date: abs(datetime.strptime(date, "%Y-%m-%d") - target_date))
    except ValueError:
        print(f"Error with expiration dates for {ticker_symbol}. Skipping.")
        continue
    
    print(f"Selected Expiration Date for {ticker_symbol}: {selected_expiration}")

    # Step 3: Get option chain for the selected expiration date
    option_chain = stock.option_chain(selected_expiration)
    put_options = option_chain.puts
    
    # Step 4: Get out-of-the-money (OTM) put options (strike price < current stock price)
    current_stock_price = stock.history(period="1d")["Close"].iloc[-1]
    print(f"Current Stock Price for {ticker_symbol}: {current_stock_price}")

    # Filter OTM puts: Strike price should be below the current stock price
    otm_puts = put_options[put_options["strike"] < current_stock_price]
    
    # Step 5: Get the 10 closest OTM strike prices
    sorted_otm_puts = otm_puts.sort_values(by="strike", ascending=False)
    closest_strikes = sorted_otm_puts.head(10)  # Select top 10 closest strikes
    
    # Add additional data for this ticker to the options_data list
    for index, row in closest_strikes.iterrows():
        options_data.append({
            "Ticker": ticker_symbol,
            "Expiration Date": selected_expiration,
            "Strike Price": row["strike"],
            "Last Price": row["lastPrice"],
            "Bid": row["bid"],
            "Ask": row["ask"],
            "Volume": row["volume"],
            "Open Interest": row["openInterest"],
            "Current Price": current_stock_price,
        })

# Convert the options data to a DataFrame
options_df = pd.DataFrame(options_data)

# Save to CSV
options_df.to_csv("options_data.csv", index=False)

# Display a snippet of the options data
print(options_df.head())

Selected Expiration Date for MSFT: 2025-02-28
Current Stock Price for MSFT: 412.2749938964844
Selected Expiration Date for AAPL: 2025-02-28
Current Stock Price for AAPL: 232.46600341796875
Selected Expiration Date for GOOGL: 2025-02-28
Current Stock Price for GOOGL: 204.80499267578125
Selected Expiration Date for AMZN: 2025-02-28
Current Stock Price for AMZN: 241.77499389648438
Selected Expiration Date for TSLA: 2025-02-28
Current Stock Price for TSLA: 386.0899963378906
Selected Expiration Date for META: 2025-02-28
Current Stock Price for META: 704.0900268554688
Selected Expiration Date for NVDA: 2025-02-28
Current Stock Price for NVDA: 118.67500305175781
Selected Expiration Date for SPY: 2025-02-28
Current Stock Price for SPY: 601.344970703125
Selected Expiration Date for V: 2025-02-28
Current Stock Price for V: 345.7699890136719
Selected Expiration Date for AMD: 2025-02-28
Current Stock Price for AMD: 117.63680267333984
Selected Expiration Date for NVDA: 2025-02-28
Current Stock Pric